In [3]:
from keras import backend
assert len(backend.tensorflow_backend._get_available_gpus()) > 0

In [2]:
import keras

Using TensorFlow backend.


In [6]:
model = keras.applications.inception_v3.InceptionV3(include_top=False, weights='imagenet', input_tensor=None, input_shape=(60,24,3), pooling=None, classes=1000)

ValueError: Input size must be at least 75x75; got `input_shape=(60, 24, 3)`

In [20]:
tf.convert_to_tensor([[1,2],[3,4]])

<tf.Tensor 'Const:0' shape=(2, 2) dtype=int32>

In [4]:
%matplotlib inline
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import random
import os

In [5]:
bitstamp = pd.read_csv('bitstampUSD_1-min_data_2012-01-01_to_2018-06-27.csv')
bitstamp.drop(labels='Timestamp', axis=1, inplace=True)
bitstamp = bitstamp[2500000:]
bitstamp.head()

,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price
2500000,609.87,609.87,609.41,609.41,4.200000,2560.068344,609.540082
2500001,609.87,609.87,609.41,609.41,4.200000,2560.068344,609.540082
2500002,609.87,609.87,609.41,609.41,4.200000,2560.068344,609.540082
2500003,610.97,610.97,610.97,610.97,0.018135,11.080002,610.970000
2500004,610.97,610.97,610.97,610.97,0.018135,11.080002,610.970000


In [ ]:
np.array.

In [ ]:
# for faster numpy concat
b = np.array(bitstamp)
a = b.reshape((-1))
n = 24*60*60
X = [a[i:i+n*7] for i in range(0,len(a) - 2*n*7,7)]

In [7]:
###code to generate y
def getY(x,n):
    x_num=len(x)
    y=[]
    for i in range(n,x_num-n):
        temp=max(x[i:i+n][1])
        temp=temp+min(x[i:i+n][2])
        temp=temp-2*x[i+n-1][3]
        temp=temp/x[i+n-1][3]
        y.append(temp)
    return y
y=getY(b,n)

In [8]:
len(X), len(y)

(733057, 733057)

In [9]:
X[0].shape

(604800,)

In [10]:
y[0]

-1.2470509412973756

In [5]:
class dataGenerator():
    def __init__(self,X,y):
        self.data_num=len(X)
        shuffled = list(zip(X,y))
        random.shuffle(shuffled)
        self.X, self.y = list(zip(*shuffled))
        self.X = list(self.X)
        self.y = list(y)
        self.iter=0
    
    def get_batch(self,n):
        temp = self.iter
        self.iter += n
        self.iter %= self.data_num
        if self.iter < temp:
            self.iter %= self.data_numss
            return np.append(self.X[temp:], self.X[:self.iter], axis = 0), np.append(self.y[temp:], self.y[:self.iter], axis = 0)
        else:
            #self.iter %= self.data_num
            return self.X[temp:self.iter], self.y[temp:self.iter]

In [6]:
for i in range(len(X)):
    X[i] = X[i].reshape(-1,7,1)

for i in range(len(y)):
    y[i] = y[i].reshape(-1)

In [7]:
#reduction: decrease in output resolution
#inception: output same dim
 
images_path = "data/MNIST_train_images.gz"
labels_path = "data/MNIST_train_labels.gz"
export_path = "saved_model/"
ckpt_path   = "tmp/ckpt"
 
#mnist = MNISTData(images_path, labels_path)
#validate_set, train_set = mnist.split(5000)

In [12]:
def main():
    tf.reset_default_graph()
    model = InceptionModel([None, 86400, 7, 1], [None, 1])
    model.train(dataGenerator(X, y), epochs=6875, keep_prob = 0.8)
    #img, label = validate_set.get_batch(85)
    #result = model.classify(img)
    #result = model.classify(generateModelTestData(2))
    result = model.classify(X)
    print(np.argmax(result, axis=1))
    print(np.argmax(y, axis=1))
    correct_prediction = np.equal(np.argmax(result, 1), np.argmax(y, 1))
    accuracy = np.sum(correct_prediction) / correct_prediction.size
    print(correct_prediction)
    print(accuracy)
    
    return

def generateModelTestData(n):
    x=np.random.uniform(-5,5,size=(n,86400*7))
    y=np.ones(n)
    return x,y

class InceptionModel():
    def __init__(self, input_shape, output_shape, ckpt_path='./', model_path='./'):
        self.ckpt_path = ckpt_path
        self.model_path = model_path
        self.num_class = output_shape[1]
        self.X = tf.placeholder(tf.float32, input_shape)
        self.y = tf.placeholder(tf.float32, output_shape)
 
        self.define_model()
        return
 
    def define_model(self):
        self.keep_prob = tf.placeholder(tf.float32)
 
        with tf.variable_scope('input'):
            _X = tf.image.resize_images(self.X, [86400, 7])
 
        with tf.variable_scope('stem'):
            _stem = stem(_X)
 
        _inception_a = {-1: _stem}
        for i in range(4):#4次运算，用字典来存放计算结果
            with tf.variable_scope('inception_a_'+str(i)):
                _inception_a[i] = inception_a(_inception_a[i-1])
 
        with tf.variable_scope('reduction_a'):
            _reduction_a = reduction_a(_inception_a[3])
      
        #another max_pool
        _first_pool = tf.nn.max_pool([_reduction_a[0]], [1, 4, 1, 1], [1, 2, 1, 1], padding='VALID', name='pool')
 
        _inception_b = {-1: _first_pool}
        for i in range(7):
            with tf.variable_scope('inception_b_'+str(i)):
                _inception_b[i] = inception_b(_inception_b[i-1])
 
        with tf.variable_scope('reduction_b'):
            _reduction_b = reduction_b(_inception_b[6])
 
        _inception_c = {-1: _reduction_b}
        for i in range(3):
            with tf.variable_scope('inception_c_'+str(i)):
                _inception_c[i] = inception_c(_inception_c[i-1])
        
        pool = tf.nn.max_pool(_inception_c[2], [1, 4, 1, 1], [1, 3, 1, 1], padding='VALID', name='pool')
        pool_f = tf.reshape(pool, [-1, 1536])
        pool_drop = tf.nn.dropout(pool_f, self.keep_prob)
        self.fc = dense(pool_drop, 'fc', 1)
        self.y_hat=dense1(tf.reshape(self.fc,[1,-1]),'fc',1)
        #self.y_hat = tf.nn.softmax(self.fc, axis = 0, name='y_hat')
        #print("softmax")
        #print(self.y_hat.shape)
        # Creates a saver object exclusively for the forward propagation subgraph
        model_variables = tf.get_collection_ref('tf.GraphKeys.MODEL_VARIABLES')
        self.model_saver = tf.train.Saver(model_variables)
        return
 
 
    def train(self, data_generator, batch_size=1, epochs=4000, keep_prob=0.8): 
        self.cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=self.y, logits=self.fc)
        self.train_step = tf.train.AdamOptimizer(1e-4).minimize(self.cross_entropy)
 
        self.correct_prediction = tf.equal(tf.argmax(self.y_hat, 1), tf.argmax(self.y, 1))
        self.accuracy = tf.reduce_mean(tf.cast(self.correct_prediction, tf.float32))
        # Creates a saver object to generate checkpoints during training. This one also saves
        # the gradients and the increment of the Adam Optimizer
        self.saver = tf.train.Saver()
        with tf.Session() as sess:
            writer = tf.summary.FileWriter("logs/", sess.graph) # graph

            sess.run(tf.global_variables_initializer())
            for i in range(epochs):
                images, labels = data_generator.get_batch(batch_size)
                train_accuracy = self.accuracy.eval(feed_dict={self.X: np.array(images).reshape(-1,n,7,1), self.y: np.array(labels).reshape(-1,1), self.keep_prob: keep_prob})
                print('step %d, training accuracy %g' % (i, train_accuracy))
                # break down
                self.train_step.run(feed_dict={self.X: images,
                                               self.y: labels,
                                               self.keep_prob: keep_prob}) 
        return
 
 
    def classify(self, image):
        with tf.Session() as sess:
            self.model_saver.restore(sess, self.model_path)
 
            y = self.y_hat.eval(feed_dict={self.X: image, self.keep_prob: 1.0})
 
        return y

def stem(tensor):
    conv_1     = conv(tensor,     'conv_1',     [3, 1, 1, 32],   [1, 2, 1, 1], padding='VALID') 
    conv_2     = conv(conv_1,     'conv_2',     [3, 1, 32, 32],  [1, 2, 1, 1], padding='VALID')
    conv_3     = conv(conv_2,     'conv_3',     [3, 1, 32, 64],  [1, 2, 1, 1])
    conv_4_1   = conv(conv_3,     'conv_4_1',   [3, 1, 64, 64],  [1, 2, 1, 1], padding='VALID')
    conv_4_2   = conv(conv_3,     'conv_4_2',   [3, 1, 64, 96],  [1, 2, 1, 1], padding='VALID')
    concat_1   = tf.concat([conv_4_1, conv_4_2], axis=3, name='concat_1')
    conv_5_1_1 = conv(concat_1,   'conv_5_1_1', [1, 1, 160, 64], [1, 1, 1, 1])
    conv_5_1_2 = conv(conv_5_1_1, 'conv_5_1_2', [3, 1, 64, 96],  [1, 2, 1, 1], padding='VALID')
    conv_5_2_1 = conv(concat_1,   'conv_5_2_1', [1, 1, 160, 64], [1, 1, 1, 1])
    #conv_5_2_2 = conv(conv_5_2_1, 'conv_5_2_2', [7, 1, 64, 64],  [1, 1, 1, 1])
    #conv_5_2_3 = conv(conv_5_2_2, 'conv_5_2_3', [1, 7, 64, 64],  [1, 1, 1, 1])
    conv_5_2_3 = conv(conv_5_2_1, 'conv_5_2_3', [1, 5, 64, 64],  [1, 1, 1, 1])
    conv_5_2_4 = conv(conv_5_2_3, 'conv_5_2_4', [3, 1, 64, 96],  [1, 2, 1, 1], padding='VALID')
    concat_2   = tf.concat([conv_5_1_2, conv_5_2_4], axis=3, name='concat_2')
    conv_6_1   = conv(concat_2,   'conv_6_1_1', [3, 1, 192, 192],  [1, 2, 1, 1], padding='VALID')
    pool_6_2   = tf.nn.max_pool(concat_2, [1, 3, 1, 1], [1, 2, 1, 1], padding='VALID', name='pool_6_2')
    concat_3   = tf.concat([conv_6_1, pool_6_2], axis=3, name='concat_3')
    return concat_3

def inception_a(tensor):
    pool_1_1 = tf.nn.avg_pool(tensor, [1, 3, 3, 1], [1, 1, 1, 1], padding='SAME', name='pool_1_1')
    conv_1_1 = conv(pool_1_1, 'conv_1_1', [1, 1, 384, 96], [1, 1, 1, 1])
    conv_2_1 = conv(tensor,   'conv_2_1', [1, 1, 384, 96], [1, 1, 1, 1])
    conv_3_1 = conv(tensor,   'conv_3_1', [1, 1, 384, 64], [1, 1, 1, 1])
    conv_3_2 = conv(conv_3_1, 'conv_3_2', [3, 1, 64, 96],  [1, 1, 1, 1])
    conv_4_1 = conv(tensor,   'conv_4_1', [1, 1, 384, 64], [1, 1, 1, 1])
    conv_4_2 = conv(conv_4_1, 'conv_4_2', [3, 1, 64, 96],  [1, 1, 1, 1])
    conv_4_3 = conv(conv_4_2, 'conv_4_3', [3, 1, 96, 96],  [1, 1, 1, 1])
    concat = tf.concat([conv_1_1, conv_2_1, conv_3_2, conv_4_3], axis=3, name='concat')
    return concat
def reduction_a(tensor):
    pool_1_1 = tf.nn.max_pool(tensor, [1, 3, 1, 1], [1, 3, 1, 1], padding='VALID', name='pool_1_1')
    conv_2_1 = conv(tensor,   'conv_2_1', [3, 1, 384, 384], [1, 3, 1, 1], padding='VALID')
    conv_3_1 = conv(tensor,   'conv_3_1', [1, 1, 384, 192], [1, 1, 1, 1])
    conv_3_2 = conv(conv_3_1, 'conv_3_2', [3, 1, 192, 224], [1, 1, 1, 1])
    conv_3_3 = conv(conv_3_2, 'conv_3_3', [3, 1, 224, 256], [1, 3, 1, 1], padding='VALID')
    concat = tf.concat([pool_1_1, conv_2_1, conv_3_3], axis=3, name='concat')
    return concat
def inception_b(tensor):
    pool_1_1 = tf.nn.avg_pool(tensor, [1, 3, 3, 1], [1, 1, 1, 1], padding='SAME', name='pool_1_1')
    conv_1_2 = conv(pool_1_1, 'conv_1_2', [1, 1, 1024, 128], [1, 1, 1, 1],)
    conv_2_1 = conv(tensor,   'conv_2_1', [1, 1, 1024, 384], [1, 1, 1, 1])
    conv_3_1 = conv(tensor,   'conv_3_1', [1, 1, 1024, 192], [1, 1, 1, 1])
    conv_3_2 = conv(conv_3_1, 'conv_3_2', [1, 3, 192, 224],  [1, 1, 1, 1])
    conv_3_3 = conv(conv_3_2, 'conv_3_3', [1, 3, 224, 256],  [1, 1, 1, 1])
    conv_4_1 = conv(tensor,   'conv_4_1', [1, 1, 1024, 192], [1, 1, 1, 1])
    conv_4_2 = conv(conv_4_1, 'conv_4_2', [1, 5, 192, 192],  [1, 1, 1, 1])
    conv_4_3 = conv(conv_4_2, 'conv_4_3', [5, 1, 192, 224],  [1, 1, 1, 1])
    conv_4_4 = conv(conv_4_3, 'conv_4_4', [1, 5, 224, 224],  [1, 1, 1, 1])
    conv_4_5 = conv(conv_4_4, 'conv_4_5', [5, 1, 224, 256],  [1, 1, 1, 1])
    concat = tf.concat([conv_1_2, conv_2_1, conv_3_3, conv_4_5], axis=3, name='concat')
    return concat
def reduction_b(tensor):
    pool_1_1 = tf.nn.max_pool(tensor, [1, 3, 1, 1], [1, 3, 1, 1], padding='VALID', name='pool_1_1')
    conv_2_1 = conv(tensor,   'conv_2_1', [1, 1, 1024, 192], [1, 1, 1, 1])
    conv_2_2 = conv(conv_2_1, 'conv_2_2', [3, 1, 192, 192],  [1, 3, 1, 1], padding='VALID')
    conv_3_1 = conv(tensor,   'conv_3_1', [1, 1, 1024, 256], [1, 1, 1, 1])
    conv_3_2 = conv(conv_3_1, 'conv_3_2', [1, 5, 256, 256],  [1, 1, 1, 1])
    conv_3_3 = conv(conv_3_2, 'conv_3_3', [5, 1, 256, 320],  [1, 1, 1, 1])
    conv_3_4 = conv(conv_3_3, 'conv_3_4', [3, 1, 320, 320],  [1, 3, 1, 1], padding='VALID')
    concat = tf.concat([pool_1_1, conv_2_2, conv_3_4], axis=3, name='concat')
    return concat
def inception_c(tensor):
    pool_1_1   = tf.nn.avg_pool(tensor, [1, 3, 3, 1], [1, 1, 1, 1], padding='SAME', name='pool_1_1')
    conv_1_2   = conv(pool_1_1, 'conv_1_2',   [1, 1, 1536, 256], [1, 1, 1, 1])
    conv_2_1   = conv(tensor,   'conv_2_1',   [1, 1, 1536, 256], [1, 1, 1, 1])
    conv_3_1   = conv(tensor,   'conv_3_1',   [1, 1, 1536, 384], [1, 1, 1, 1])
    conv_3_2_1 = conv(conv_3_1, 'conv_3_2_1', [1, 3, 384, 256],  [1, 1, 1, 1])
    conv_3_2_2 = conv(conv_3_1, 'conv_3_2_2', [3, 1, 384, 256],  [1, 1, 1, 1])
    conv_4_1   = conv(tensor,   'conv_4_1',   [1, 1, 1536, 384], [1, 1, 1, 1])
    conv_4_2   = conv(conv_4_1, 'conv_4_2',   [1, 3, 384, 448],  [1, 1, 1, 1])
    conv_4_3   = conv(conv_4_2, 'conv_4_3',   [3, 1, 448, 512],  [1, 1, 1, 1])
    conv_4_3_1 = conv(conv_4_3, 'conv_4_3_1', [1, 3, 512, 256],  [1, 1, 1, 1])
    conv_4_3_2 = conv(conv_4_3, 'conv_4_3_2', [3, 1, 512, 256],  [1, 1, 1, 1])
    concat = tf.concat([conv_1_2, conv_2_1, conv_3_2_1, conv_3_2_2, conv_4_3_1, conv_4_3_2], axis=3, name='concat')
    return concat

def conv(tensor, name, shape, strides=[1, 1, 1, 1], padding='SAME', activation=tf.nn.relu):
    W = tf.get_variable(name+"_W", shape)
    b = tf.get_variable(name+"_b", shape[-1])
    tf.add_to_collection('tf.GraphKeys.MODEL_VARIABLES', W)
    tf.add_to_collection('tf.GraphKeys.MODEL_VARIABLES', b)
    z = tf.nn.conv2d(tensor, W, strides=strides, padding=padding, name=name+'_z')
    h = tf.add(z, b, name=name+'_h')
    a = activation(h, name=name+'_a') 
    return a
def dense(tensor, name, num_out):
    W_fc = tf.get_variable('W_fc', [tensor.shape[1], num_out])
    b_fc = tf.get_variable('b_fc', [num_out])
    tf.add_to_collection('tf.GraphKeys.MODEL_VARIABLES', W_fc)
    tf.add_to_collection('tf.GraphKeys.MODEL_VARIABLES', b_fc)
    z_fc = tf.matmul(tensor, W_fc, name='z_fc')
    h_fc = tf.add(z_fc, b_fc, name='h_fc')
    return h_fc
def dense1(tensor, name, num_out):
    W_fc = tf.get_variable('W_fc1', [tensor.shape[1], num_out])
    b_fc = tf.get_variable('b_fc1', [num_out])
    tf.add_to_collection('tf.GraphKeys.MODEL_VARIABLES', W_fc)
    tf.add_to_collection('tf.GraphKeys.MODEL_VARIABLES', b_fc)
    z_fc = tf.matmul(tensor, W_fc, name='z_fc1')
    h_fc = tf.add(z_fc, b_fc, name='h_fc1') 
    return h_fc

In [ ]:
main()

stem
(?, 1349, 7, 384)
inception_a
(?, 1349, 7, 384)
inception_a
(?, 1349, 7, 384)
inception_a
(?, 1349, 7, 384)
inception_a
(?, 1349, 7, 384)
reduction_a
(?, 449, 7, 1024)
inception_b
(1, 223, 7, 1024)
inception_b
(1, 223, 7, 1024)
inception_b
(1, 223, 7, 1024)
inception_b
(1, 223, 7, 1024)
inception_b
(1, 223, 7, 1024)
inception_b
(1, 223, 7, 1024)
inception_b
(1, 223, 7, 1024)
reduction_b
(1, 74, 7, 1536)
inception_c
(1, 74, 7, 1536)
inception_c
(1, 74, 7, 1536)
inception_c
(1, 74, 7, 1536)
max_pool
(168, 1536)
dropout
(168, 1536)
dense1
(168, 1)
dense2
(1, 1)
defined
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

step 0, training accuracy 1


In [1]:
import tensorflow as tf
tf.test.is_gpu_available()

True

In [2]:
tf.test.is_built_with_cuda()

True

In [36]:
del tf

In [37]:
import tensorflow as tf

In [38]:
tensorflow.__file__

'/anaconda3/lib/python3.6/site-packages/tensorflow/__init__.py'

In [34]:
import sys

In [35]:
sys.path

['',
 '/anaconda3/lib/python36.zip',
 '/anaconda3/lib/python3.6',
 '/anaconda3/lib/python3.6/lib-dynload',
 '/anaconda3/lib/python3.6/site-packages',
 '/anaconda3/lib/python3.6/site-packages/IPython/extensions',
 '/home/felixyan29_gmail_com/.ipython']